# Notebook test des différentes méthodes de sampling pour le ngram


In [1]:
!pip install kaggle
!pip install tiktoken
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [24]:
!pip install -U nltk==3.8.1.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.4
    Uninstalling nltk-3.4:
      Successfully uninstalled nltk-3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import pandas as pd
import numpy as np
import random
import nltk
import tensorflow as tf
import matplotlib.pyplot as plt
import tiktoken
import os
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
from collections import Counter, defaultdict
import torch
from torch.nn import functional as F

2024-04-17 16:14:23.358197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 16:14:23.358340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 16:14:23.544279: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

On fait un preprocessing des données

In [6]:
skip_func = lambda x: x % 30 != 0
def split_data(data: pd.DataFrame):
    """
    Split the data in a training and test set.
    The training set is composed of 80% of the data.
    The test set is composed of 20% of the data.
    Useful links: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7
    """
    train_set = data.sample(frac=0.8, random_state=0)
    validation_set = data.drop(train_set.index)
    return train_set, validation_set
def import_data_n_gram(file_path: str):
    """
    Preprocess the data for the n-gram model.
    The transformation effectuated are:
    - Keep only the english songs and remove the language column
    - Remove the NaN values
    - Replace the square brackets by an empty string to remove the annotations and metadata
    - Keep only the songs with a title length inferior to 50 characters
    - Keep only the songs with a title composed of ASCII characters
    - Concatenate the title and the lyrics in a single column
    - Transform the text in lowercase
    - Split the data in a training and validation set
    Useful links: https://necromuralist.github.io/Neurotic-Networking/posts/nlp/n-gram-pre-processing/index.html
    """
    data = pd.read_csv(file_path, usecols=['title', 'lyrics', 'language_cld3'], skiprows=lambda x: skip_func(x))
    data = data.loc[data['language_cld3'] == 'en']
    data = data.drop(columns=['language_cld3'])
    data = data.dropna()
    data['lyrics'] = data['lyrics'].str.replace(r"\[.*\]", "")
    data = data[data['title'].apply(lambda x: len(x) < 50)]
    data = data[data['title'].str.match(r'^[\x00-\x7F]*$')]
    data['title_lyrics'] = data['title'] + ' ' + data['lyrics']
    data[['title_lyrics', 'title', 'lyrics']] = data[['title_lyrics', 'title', 'lyrics']].apply(lambda x: x.str.lower())
    train_set, validation_set = split_data(data)
    return train_set, validation_set
X_train, X_test = import_data_n_gram('/kaggle/input/dataset-genius/song_lyrics.csv')
X_train = X_train['lyrics']
X_test = X_test['lyrics']


27255    you know i can't take it anymore\ntake me out ...
40372    [verse]\nthe fire makes the city look so brigh...
45239    [verse 1]\nlast night's party\ndidn't go as pl...
11669    we crawled night and day through the tears and...
41631    this connection we got girl is something deepe...
                               ...                        
18084    [verse 1]\nup so high, got so tied\nyou got me...
18679    [round 1: aftershock]\ni said you mans with my...
3439     [verse 1]\ntoday what's day is night\nsomeday ...
16479    [verse 1]\nfirst light breaks in the morning s...
25463    [verse 1]\nyou didn't spark no revolution\nbet...
Name: lyrics, Length: 26034, dtype: object

On réutilise les tokenizers du cours

In [7]:
#Define tokenizers from course
def lemma_tokenize(doc):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(t) for t in word_tokenize(doc)]

def char_tokenize(doc):
    return [char for char in doc]

def byte_tokenize(doc):
    tokens = doc.encode("utf-8")
    tokens = list(map(int, tokens))
    return [str(token) for token in tokens]

def gpt_tokenize(doc):
    enc = tiktoken.encoding_for_model("gpt-4")
    tokens = enc.encode(doc)
    return [str(token) for token in tokens]

Calcul des probabilités du ngram, elles nous serviront à générer les mots.

In [8]:
def tokenize(text):
    """Tokenize the input text."""

    return word_tokenize(text)

def count_ngrams(tokens, n):
    """Counts n-grams."""

    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

    return Counter(ngrams)

def calculate_ngram_probabilities(train_tokens, n, test_tokens, k=0.00001):
    """Calculates n-gram probabilities."""

    vocab = set(train_tokens)
    V = len(vocab)
    ngram_counts = count_ngrams(train_tokens, n)
    n_minus_one_gram_counts = count_ngrams(train_tokens, n-1)
    ngram_probabilities = defaultdict(float)

    for ngram in ngram_counts:
        prefix = ngram[:-1]
        ngram_counts[ngram] += k
        n_minus_one_gram_counts[prefix] += k
        ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)

    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        if ngram not in ngram_counts:
            ngram_counts[ngram] = k
            prefix = ngram[:-1]
            if prefix not in n_minus_one_gram_counts:
                n_minus_one_gram_counts[prefix] = k
            ngram_probabilities[ngram] = (ngram_counts[ngram] + k) / (n_minus_one_gram_counts[prefix] + k*V)

    return ngram_probabilities

Calcul des probabilites pour un 5-gram en tokenizant à l'aide de gpt

In [ ]:
X_train_gpt = [gpt_tokenize(text) for text in X_train]
X_test_gpt = [gpt_tokenize(text) for text in X_test]
n = 5

train_tokens = word_tokenize(" ".join(X_train))
vocab = set(train_tokens)
test_tokens = word_tokenize(" ".join(X_test))
ngram_probabilities = calculate_ngram_probabilities(train_tokens, n, test_tokens, k=0.00001)
ngrams_counts = count_ngrams(train_tokens, n)

In [1]:
ngram_probabilities


NameError: name 'ngram_probabilities' is not defined

Calcul de perplexité qui va nous permettre de définir la précision du modèle, plus on augmente le n, ,plus la perplexité du set d'entrainement diminue et celle du set de test augmente

In [27]:
def calculate_perplexity(test_tokens, ngram_probabilities, n):
    """Calculates the perplexity of a test corpus given n-gram probabilities."""
    log_probability_sum = 0
    ngram_count = 0

    for i in range(len(test_tokens)-n+1):
        ngram = tuple(test_tokens[i:i+n])
        log_probability_sum += math.log2(ngram_probabilities[ngram])
        ngram_count += 1

    average_log_probability = -log_probability_sum / ngram_count
    perplexity = math.pow(2, average_log_probability)

    return perplexity

In [28]:
calculate_perplexity(train_tokens, ngram_probabilities, n)

17.03522341729249

In [29]:
calculate_perplexity(test_tokens, ngram_probabilities, n)

2745.1074048237683

On va générer les mots à l'aide de plusieurs méthode de sampling, d'abord la méthode gloutonne vu en cours, ensuite la méthode top-k et enfin la méthode top-p

In [50]:
def greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 50):

    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]

    for i in range(max_length):

        probs = dict()

        for v in vocab:

            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities[ngram]

        best_token = max(probs, key=probs.get) # greedy
        #print(best_v)
        #print(probs[best_v])

        if probs[best_token] == 0:
            print("prob = 0")
            return sentence

        sentence.append(best_token)
        context = list(context)[1:]
        context.append(best_token)
        context = tuple(context)

    return sentence
    import random

def top_k_sampling(context, vocab, ngram_probabilities, n, k, max_length=50):
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]

    for i in range(max_length):
        probs = dict()

        for v in vocab:
            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities.get(ngram, 0)

        top_k_tokens = sorted(probs, key=probs.get, reverse=True)[:k]

        if sum(probs[token] for token in top_k_tokens) == 0:
            print("All top-k probs are 0")
            return sentence

        next_token = random.choices(top_k_tokens, weights=[probs[token] for token in top_k_tokens])[0]

        sentence.append(next_token)
        context = list(context)[1:]
        context.append(next_token)
        context = tuple(context)

    return sentence
def top_p_sampling(context, vocab, ngram_probabilities, n, p, max_length=50):
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]

    for i in range(max_length):
        probs = dict()

        for v in vocab:
            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities.get(ngram, 0)

        sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
        cumulative_probs = []
        cumulative_sum = 0

        for token, prob in sorted_probs:
            cumulative_sum += prob
            cumulative_probs.append((token, cumulative_sum))
            if cumulative_sum >= p:
                break

        if cumulative_sum == 0:
            print("All probs are 0")
            return sentence

        next_token = random.choices([t for t, _ in cumulative_probs],
                                    weights=[prob for _, prob in cumulative_probs])[0]

        sentence.append(next_token)
        context = list(context)[1:]
        context.append(next_token)
        context = tuple(context)

    return sentence
def top_k_top_p_sampling(context, vocab, ngram_probabilities, n, k, p, max_length=50):
    sentence = []

    if len(context) < (n-1):
        print("len(context) < n")
        return sentence

    context = context[-(n-1):]

    for i in range(max_length):
        probs = dict()

        for v in vocab:
            ngram = list(context)
            ngram.append(v)
            ngram = tuple(ngram)
            probs[v] = ngram_probabilities.get(ngram, 0)

        top_k_tokens = sorted(probs, key=probs.get, reverse=True)[:k]
        
        top_k_probs = {token: probs[token] for token in top_k_tokens}
        
        sorted_probs = sorted(top_k_probs.items(), key=lambda x: x[1], reverse=True)
        cumulative_probs = []
        cumulative_sum = 0

        for token, prob in sorted_probs:
            cumulative_sum += prob
            cumulative_probs.append((token, cumulative_sum))
            if cumulative_sum >= p:
                break

        if cumulative_sum == 0:
            print("All top-k probs are 0")
            return sentence

        next_token = random.choices([t for t, _ in cumulative_probs], 
                                    weights=[prob for _, prob in cumulative_probs])[0]

        sentence.append(next_token)
        context = list(context)[1:]
        context.append(next_token)
        context = tuple(context)

    return sentence

In [55]:
context = ['you',
              'believe',
              'in',
              'something',
              'good']
#context = ['the', 'one']
#sentence = greedy_sampling(context, vocab, ngram_probabilities, n, max_length = 200)
sentence = top_p_sampling(context, vocab, ngram_probabilities, n, 0.4, max_length = 200)
#sentence = top_k_top_p_sampling(context, vocab, ngram_probabilities, n, 5,0.2, max_length = 200)
print(" ".join(context) + " " +  " ".join(sentence))

All probs are 0
you believe in something good ? do you feel like movin ' movin' do you feel like movin ' movin' do you feel like movin ' movin' do you feel like movin ' movin' do you feel like yourself tonight ? ( do you feel ? ) do you wan na know why ? 'cause in bro's
